In [1]:
import requests
from urllib.parse import unquote
import pandas as pd
from datetime import datetime



def get_options(ticker):

    try :
        geturl=r'https://www.barchart.com/stocks/quotes/AAPL/options'
        apiurl=r'https://www.barchart.com/proxies/core-api/v1/options/get'

        getheaders={

            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'cache-control': 'max-age=0',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
            }

        getpay={
            'page': 'all'
        }

        s=requests.Session()
        r=s.get(geturl,params=getpay, headers=getheaders)

        headers={
            'accept': 'application/json',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36',
            'x-xsrf-token': unquote(unquote(s.cookies.get_dict()['XSRF-TOKEN']))
        }

        payload={
                'fields': 'symbol,baseSymbol,strikePrice,lastPrice,theoretical,volatility,delta,gamma,theta,vega,rho,volume,openInterest,volumeOpenInterestRatio,optionType,daysToExpiration,expirationDate,tradeTime,averageVolatility,historicVolatility30d,baseNextEarningsDate',
                "baseSymbol": str(ticker),
                "groupBy":"optionType",
                "expirationDate":"",
                "meta":"field.shortName,expirations",
                "orderBy":"strikePrice",
                "orderDir":"asc",
                "limit" : 10000
            }
            

        r=s.get(apiurl,params=payload, headers=headers)
        j=r.json()


        # Define DataFrame containing Put and Call option price
        df_call = pd.DataFrame(j['data']['Call'])
        df_put = pd.DataFrame(j['data']['Put'])
        df_merged = pd.concat([df_call, df_put], axis=0)
        df_merged['ExtractTime'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

        return df_merged
    
    except:
        # Create a function to save the ticker that generates an error
        # for the moment we only pass
        pass
    
value = get_options(ticker="GOOGL")
value


In [ ]:
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown
from functools import lru_cache

class OptionGreeks:
    """A class for calculating and visualizing option Greeks using QuantLib."""
    
    def __init__(self, option_type, spot, strike, volatility, risk_free_rate, dividend_yield, time_to_maturity):
        """Initialize the option pricing environment with the provided parameters."""
        self.option_type = option_type
        self.spot = spot
        self.strike = strike
        self.volatility = volatility
        self.risk_free_rate = risk_free_rate
        self.dividend_yield = dividend_yield
        self.time_to_maturity = time_to_maturity
        
        # Set up QuantLib environment - use constants for efficiency
        self._setup_environment()
        
    def _setup_environment(self):
        """Set up the QuantLib pricing environment."""
        # Core environment setup
        self.calendar = ql.TARGET()
        self.day_count = ql.Actual365Fixed()
        self.settlement_date = ql.Date.todaysDate()
        ql.Settings.instance().evaluationDate = self.settlement_date
        
        # Option parameters
        self.maturity_date = self.settlement_date + ql.Period(int(self.time_to_maturity * 365), ql.Days)
        option_type = ql.Option.Call if self.option_type == 'call' else ql.Option.Put
        self.payoff = ql.PlainVanillaPayoff(option_type, self.strike)
        self.exercise = ql.EuropeanExercise(self.maturity_date)
        self.option = ql.VanillaOption(self.payoff, self.exercise)
        
        # Market data handles
        self.spot_quote = ql.SimpleQuote(self.spot)
        self.spot_handle = ql.QuoteHandle(self.spot_quote)
        self.rate_ts = ql.FlatForward(
            self.settlement_date, 
            self.risk_free_rate, 
            self.day_count
        )
        self.rate_handle = ql.YieldTermStructureHandle(self.rate_ts)
        
        self.dividend_ts = ql.FlatForward(
            self.settlement_date, 
            self.dividend_yield, 
            self.day_count
        )
        self.dividend_handle = ql.YieldTermStructureHandle(self.dividend_ts)
        
        self.vol_ts = ql.BlackConstantVol(
            self.settlement_date, 
            self.calendar, 
            self.volatility, 
            self.day_count
        )
        self.volatility_handle = ql.BlackVolTermStructureHandle(self.vol_ts)
        
        # Black-Scholes-Merton process
        self.bsm_process = ql.BlackScholesMertonProcess(
            self.spot_handle, 
            self.dividend_handle, 
            self.rate_handle, 
            self.volatility_handle
        )
        
        # Set pricing engine
        engine = ql.AnalyticEuropeanEngine(self.bsm_process)
        self.option.setPricingEngine(engine)
    
    def update_spot(self, spot):
        """Update the spot price."""
        self.spot_quote.setValue(spot)
    
    @property
    def price(self):
        """Calculate the option price."""
        return self.option.NPV()
    
    @property
    def greeks(self):
        """Calculate all option Greeks."""
        return {
            'delta': self.option.delta(),
            'gamma': self.option.gamma(),
            'vega': self.option.vega(),
            'rho': self.option.rho(),
            'theta': self.option.theta()
        }
    
    def plot_greek(self, greek_name):
        """Plot a specific Greek against spot price."""
        # Define range of spot prices (50% below to 50% above current spot)
        spot_range = np.linspace(0.5 * self.spot, 1.5 * self.spot, 100)
        greek_values = []
        
        # Calculate the Greek for each spot price
        for spot_price in spot_range:
            self.update_spot(spot_price)
            greek_values.append(self.greeks[greek_name])
        
        # Create plot
        plt.figure(figsize=(10, 6))
        plt.plot(spot_range, greek_values, linewidth=2)
        plt.axvline(x=self.strike, color='r', linestyle='--', alpha=0.5, label='Strike Price')
        
        plt.xlabel('Spot Price', fontsize=12)
        plt.ylabel(f'{greek_name.capitalize()}', fontsize=12)
        plt.title(f'{greek_name.capitalize()} vs Spot Price for {self.option_type.capitalize()} Option', fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # Add option parameters as text
        param_text = (
            f"Strike: {self.strike}\n"
            f"Volatility: {self.volatility:.2f}\n"
            f"Risk-free rate: {self.risk_free_rate:.2f}\n"
            f"Dividend yield: {self.dividend_yield:.2f}\n"
            f"Time to maturity: {self.time_to_maturity:.1f} years"
        )
        plt.annotate(param_text, xy=(0.02, 0.05), xycoords='axes fraction', 
                     bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        # Reset spot price to original value
        self.update_spot(self.spot)

# Create interactive widget with improved UI
def interactive_option_analyzer(
    spot=100, 
    strike=100, 
    volatility=0.2, 
    risk_free_rate=0.05, 
    dividend_yield=0.02, 
    time_to_maturity=1.0, 
    option_type='call', 
    greek_type='delta'
):
    """Interactive function for option Greeks visualization with improved parameters."""
    
    calculator = OptionGreeks(
        option_type, spot, strike, volatility, 
        risk_free_rate, dividend_yield, time_to_maturity
    )
    
    # Display option price
    price = calculator.price
    print(f"Option Price: ${price:.2f}")
    
    # Display the selected Greek value at current spot
    greek_value = calculator.greeks[greek_type]
    print(f"Current {greek_type.capitalize()}: {greek_value:.6f}")
    
    # Plot the selected Greek
    calculator.plot_greek(greek_type)

# Create enhanced interactive widget
interact(
    interactive_option_analyzer,
    spot=FloatSlider(min=50, max=150, step=1, value=100, description='Spot Price:'),
    strike=FloatSlider(min=50, max=150, step=1, value=100, description='Strike Price:'),
    volatility=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Volatility:'),
    risk_free_rate=FloatSlider(min=0.0, max=0.1, step=0.005, value=0.05, description='Risk-Free Rate:'),
    dividend_yield=FloatSlider(min=0.0, max=0.1, step=0.005, value=0.02, description='Dividend Yield:'),
    time_to_maturity=FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0, description='Time (years):'),
    option_type=Dropdown(options=['call', 'put'], value='call', description='Option Type:'),
    greek_type=Dropdown(
        options=['delta', 'gamma', 'vega', 'rho', 'theta'], 
        value='delta', 
        description='Greek:'
    )
)

In [35]:
URL = "https://symbol-browser.aws.barchart.com/v1/search/instruments"

HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
}
PARAMS = {
    "application":5,
    "query":"A",
    "size":100,
    # "region":"US",
    # "types":"5,7,8,11,15,9,1,3,4,6,23"
}
s=requests.Session()
r=s.get(URL,params=PARAMS, headers=HEADERS)
j=r.json()
data = j['result']['symbols']
instruments = [item['instrument'] for item in data]
df = pd.DataFrame(instruments)
df
# df


KeyError: 'instrument'